## Linear Model

In [ ]:
#| default_exp linear_model

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
#| export
from __future__ import annotations
from jax_interpret.imports import *

In [ ]:
def l2_loss(x1, x2, weights=None):
    if weights is None:
        return jnp.mean(jnp.square(x1 - x2)) / 2.0
    else:
        return jnp.sum((weights / jnp.linalg.norm(weights, ord=1)) * jnp.square(x1 - x2)) / 2.0

In [ ]:
def _init_train_fn(
    X: jnp.ndarray, # Input data
    y: jnp.ndarray, # Target data
    fit_bias: bool = True, # Fit bias term
    seed: int = 42, # Random seed
):  
    rng = jax.random.PRNGKey(seed)
    n_samples, n_features = X.shape
    rng, w_key, b_key = jax.random.split(rng, 3)
    w = jax.random.normal(w_key, (n_features,))
    if fit_bias:
        b = jax.random.normal(b_key, (1,))
    else:
        b = jnp.zeros(1)
    params = dict(w=w, b=b)
    return params

def calculate_loss(
    params: Dict[str, jnp.ndarray],
    batch: Tuple[jnp.ndarray, jnp.ndarray],
    loss_fn: Callable,
    weights: jnp.ndarray = None,
    reg_term: int = None,
    alpha: float = 1.0
):
    """Calculate the loss for a batch of data."""
    w, b = params["w"], params["b"]
    X, y = batch
    y_pred = jnp.dot(X, w) + b
    loss = loss_fn(y, y_pred, weights)
    if reg_term is not None:
        reg = jnp.linalg.norm(w, ord=reg_term)
        loss += jnp.mean(reg) * alpha
    return loss

def sgd_train_linear_model(
    X: jnp.ndarray, # Input data. Shape: `(N, k)`
    y: jnp.ndarray, # Target data. Shape: `(N,)` or `(N, 1)`
    weights: jnp.ndarray = None, # Initial weights. Shape: `(k,)`
    lr: float = 0.01, # Learning rate
    n_epochs: int = 100, # Number of epochs
    batch_size: int = 32, # Batch size
    seed: int = 42, # Random seed
    loss_fn: Callable = l2_loss, # Loss function
    reg_term: int = None, # Regularization term
    alpha: float = 1.0, # Regularization strength
    fit_bias: bool = True, # Fit bias term
) -> Tuple[np.ndarray, np.ndarray]: # The trained weights and bias
    """Train a linear model using SGD."""

    @jax.jit
    def sgd_step(params, opt_state, batch):
        """Perform a single SGD step."""
        grads = jax.grad(calculate_loss)(params, batch, loss_fn, weights, reg_term, alpha)
        updates, opt_state = opt.update(grads, opt_state)
        params = optax.apply_updates(params, updates)
        return params, opt_state

    # TODO: Check shapes of X and y
    n_samples = X.shape[0]
    params = _init_train_fn(X, y, fit_bias, seed)
    opt = optax.sgd(lr)
    opt_state = opt.init(params)
    for epoch in range(n_epochs):
        for i in range(0, n_samples, batch_size):
            X_batch = X[i : i + batch_size]
            y_batch = y[i : i + batch_size]
            params, opt_state = sgd_step(params, opt_state, (X_batch, y_batch))
    return params["w"], params["b"]


In [ ]:
#| export
class BaseEstimator:
    def __init__(self):
        ...

    def fit(self, X, y):
        ...

In [ ]:
# export
class LinearModel(BaseEstimator):
    def __init__(
        self,
        bias: bool = True,
        trainer_fn: Callable=None,
        **kwargs,
    ):
        self.fit_bias = bias
        self.trainer_fn = sgd_train_linear_model if trainer_fn is None else trainer_fn
    
    def fit(
        self, 
        X: jnp.ndarray, 
        y: jnp.ndarray,
        weights: jnp.ndarray = None,
        **kwargs,
    ) -> LinearModel:
        self.coef_, self.bias_ = self.trainer_fn(
            X, y, weights, fit_bias=self.fit_bias, **kwargs)
        return self

In [ ]:
#| export
class Lasso(LinearModel):
    def __init__(self, alpha: float = 1.0, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha

    def fit(self, X: jnp.ndarray, y: jnp.ndarray, weights: jnp.ndarray = None, **kwargs) -> LinearModel:
        return super().fit(X, y, weights, reg_term=1, alpha=self.alpha, **kwargs)

In [ ]:
#| export
class Ridge(LinearModel):
    def __init__(self, alpha: float = 1.0, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha

    def fit(self, X: jnp.ndarray, y: jnp.ndarray, weights: jnp.ndarray = None, **kwargs) -> LinearModel:
        return super().fit(X, y, weights, reg_term=2, alpha=self.alpha, **kwargs)

#### Test 

In [ ]:
from sklearn.datasets import make_regression

In [ ]:
X, y = make_regression(n_samples=500, n_features=20)

In [ ]:
lm = LinearModel()
lm.fit(X, y)

<__main__.LinearModel>

In [ ]:
lasso = Lasso(alpha=0.1)
lasso.fit(X, y)

<__main__.Lasso>

In [ ]:
ridge = Ridge(alpha=0.1)
ridge.fit(X, y)

<__main__.Ridge>